In [1]:
import numpy as np
import itertools
import functools
from collections import Counter

In [2]:
buttons_directional_pad = {
    (0, 2): "A",
    (0, 1): "^",
    (1, 0): "<",
    (1, 1): "v",
    (1, 2): ">",
}
buttons_numerical_pad = {
    (0, 0): "7",
    (0, 1): "8",
    (0, 2): "9",
    (1, 0): "4",
    (1, 1): "5",
    (1, 2): "6",
    (2, 0): "1",
    (2, 1): "2",
    (2, 2): "3",
    (3, 1): "0",
    (3, 2): "A",
}


def get_loc_per_button(buttons):
    return {button: np.array(loc) for loc, button in buttons.items()}


loc_numeric = get_loc_per_button(buttons_numerical_pad)
loc_directional = get_loc_per_button(buttons_directional_pad)

In [3]:
def find_one_dimensional_route(start, end):
    vertical_diff, horizontal_diff = end - start
    if vertical_diff == 0:
        if horizontal_diff < 0:
            return "<" * abs(horizontal_diff)
        else:
            return ">" * abs(horizontal_diff)
    if horizontal_diff == 0:
        if vertical_diff < 0:
            return "^" * abs(vertical_diff)
        else:
            return "v" * abs(vertical_diff)
    raise ValueError("Not 1 dimensional!")


def find_shortest_routes(start, end, available):
    available = [tuple(field) for field in available]
    shortest_routes = []
    vertical_diff, horizontal_diff = end - start
    if (vertical_diff == 0) or (horizontal_diff == 0):
        return [find_one_dimensional_route(start, end) + "A"]

    corners = [start + (0, horizontal_diff), start + (vertical_diff, 0)]
    corners = [corner for corner in corners]
    for corner in corners:

        if tuple(corner) in available:
            route = (
                find_one_dimensional_route(start, corner)
                + find_one_dimensional_route(corner, end)
                + "A"
            )
            shortest_routes.append(route)
    return shortest_routes


def find_shortest_routes_between_buttons(first_place, second_place, pad):
    if pad == "numeric":
        loc = loc_numeric
    if pad == "directional":
        loc = loc_directional
    first_loc = loc[first_place]
    second_loc = loc[second_place]

    routes = find_shortest_routes(first_loc, second_loc, loc.values())
    return routes


find_shortest_routes_between_buttons("A", "4", "numeric")
find_shortest_routes_between_buttons("3", "4", "numeric")

['<<^A', '^<<A']

In [4]:
buttons_numerical_pad.values()

dict_values(['7', '8', '9', '4', '5', '6', '1', '2', '3', '0', 'A'])

In [5]:
def create_all_shortest_routes_numeric():
    shortest_route = {}
    for first_place in buttons_numerical_pad.values():
        for second_place in buttons_numerical_pad.values():
            if first_place == second_place:
                continue
            shortest_route[first_place + second_place] = (
                find_shortest_routes_between_buttons(
                    first_place, second_place, "numeric"
                )
            )
    return shortest_route


def create_all_shortest_routes_directional():
    shortest_route = {}
    for first_place in buttons_directional_pad.values():
        for second_place in buttons_directional_pad.values():
            shortest_route[first_place + second_place] = (
                find_shortest_routes_between_buttons(
                    first_place, second_place, "directional"
                )
            )
    return shortest_route


sequence_of_button_pushes_numeric = create_all_shortest_routes_numeric()
sequence_of_button_pushes_directional = create_all_shortest_routes_directional()

In [6]:
def find_button_combinations(code, shortest_route: dict):
    starting_point = "A"
    code = starting_point + code
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]


def find_button_combinations_codes(codes, shortest_route: dict):
    result = []
    for code in codes:
        result.extend(find_button_combinations(code, shortest_route))
    return result


def find_min_len_buttons_second_bot(code):
    first_directional_bot = find_button_combinations(
        code, sequence_of_button_pushes_numeric
    )
    second_directional_bot = find_button_combinations_codes(
        first_directional_bot, sequence_of_button_pushes_directional
    )
    human_moves = find_button_combinations_codes(
        second_directional_bot, sequence_of_button_pushes_directional
    )
    return min(len(buttons) for buttons in human_moves)

In [7]:
def calculate_complexity(code):
    len_buttons = find_min_len_buttons_second_bot(code)
    complexity = len_buttons * int(code[:-1])
    # print(f"{len_buttons} * {int(code[:-1])}")  # Check examples
    return complexity


codes = ["279A", "341A", "459A", "540A", "085A"]
sum(calculate_complexity(code) for code in codes)

123096

In [8]:
def find_button_presses(code, start_location):
    directions = {  # This is NOT the location of the direction on the pad, this is the acutal direction
        "^": np.array([-1, 0]),
        ">": np.array([0, 1]),
        "v": np.array([1, 0]),
        "<": np.array([0, -1]),
    }
    output = []
    current_location = start_location
    for move in code:
        if move == "A":
            output.append(current_location)
        else:
            current_location = tuple(current_location + directions[move])
    return output


def run_code(code, mode):
    if mode == "directional":
        button_locations = buttons_directional_pad
        start_position = (0, 2)
    elif mode == "numeric":
        button_locations = buttons_numerical_pad
        start_position = (3, 2)
    pressed_buttons = find_button_presses(code, start_position)
    return "".join(button_locations[location] for location in pressed_buttons)


total_string = "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
directional = run_code(total_string[12:35], "directional")
directional2 = run_code(directional, "directional")
numeric = run_code(directional2, "numeric")
numeric

'4'

# Part 2

In [9]:
def get_transitions(string):
    return [string[i] + string[i + 1] for i in range(len(string) - 1)]


def get_transitions_one_layer_up(sequence_of_buttons):
    transitions_layer_up = {}
    for transition, buttons in sequence_of_buttons.items():
        transition_result = []
        for possibility in buttons:
            transitions = get_transitions(possibility)
            transition_result.append((Counter(transitions), possibility[0]))
        transitions_layer_up[transition] = transition_result
    return transitions_layer_up


up_transitions_directional = get_transitions_one_layer_up(
    sequence_of_button_pushes_directional
)

In [10]:
@functools.cache
def n_buttons_n_directionals_up(transition, n_machines):
    """Only for directional pad"""
    if n_machines == 0:
        # All these lengths are equal by definition
        return len(sequence_of_button_pushes_directional[transition][0])
    possible = []
    options = up_transitions_directional[transition]
    for option, first_move in options:
        this_score = n_buttons_n_directionals_up("A" + first_move, n_machines - 1)
        for trans, n in option.items():
            this_score += n * n_buttons_n_directionals_up(trans, n_machines - 1)
        possible.append(this_score)
    return min(possible)


def find_button_combinations_shortest(code, shortest_route: dict):
    button_pushes = []
    for current, next in zip(code[:-1], code[1:]):
        button_pushes.append(shortest_route[current + next])
    return ["".join(option) for option in itertools.product(*button_pushes)]


def buttons_n_machines_up(code, n_machines):
    numeric_bot = find_button_combinations_shortest(
        "A" + code, sequence_of_button_pushes_numeric
    )
    transition_options = [
        (Counter(get_transitions(option)), option[0]) for option in numeric_bot
    ]

    possible = []
    for option, first_move in transition_options:
        this_score = n_buttons_n_directionals_up("A" + first_move, n_machines - 1)
        for trans, n in option.items():
            this_score += n * n_buttons_n_directionals_up(trans, n_machines - 1)
        possible.append(this_score)
    return min(possible)

In [11]:
def test(output, expected):
    if output != expected:
        raise ValueError(f"Failed test! {expected=}, {output=}")


def run_tests():
    test(n_buttons_n_directionals_up("A^", 0), 2)
    test(n_buttons_n_directionals_up("^<", 0), 3)
    test(n_buttons_n_directionals_up("^A", 1), 4)
    test(n_buttons_n_directionals_up("A>", 1), 6)
    test(n_buttons_n_directionals_up("A>", 2), 16)

    test(buttons_n_machines_up("3", 2), 12)
    test(buttons_n_machines_up("37", 2), 35)
    test(buttons_n_machines_up("029A", 2), 68)
    test(buttons_n_machines_up("980A", 2), 60)
    test(buttons_n_machines_up("179A", 2), 68)
    test(buttons_n_machines_up("456A", 2), 64)
    test(buttons_n_machines_up("379A", 2), 64)
    print("Tests succesful")


run_tests()

Tests succesful


In [12]:
codes = ["279A", "341A", "459A", "540A", "085A"]


def calculate_complexity(code):
    len_buttons = buttons_n_machines_up(code, 25)
    complexity = len_buttons * int(code[:-1])
    # print(f"{len_buttons} * {int(code[:-1])}")
    return complexity


codes = ["279A", "341A", "459A", "540A", "085A"]
sum(calculate_complexity(code) for code in codes)

154517692795352